In [2]:
# 기능구현
# 파일추가->리스트에 집어넣고, 선택된 항목 삭제하고, 저장경로 기능 까지
import os
import tkinter.ttk as ttk
import tkinter.messagebox as msgbox
from tkinter import * #all부분에 정의하지 않으면 (서브모듈은) 별도의 import 필요
from tkinter import filedialog  #파일다이얼로그 가져다 쓰기
from PIL import Image


root = Tk()
root.title("나도 코딩! 사진 이어붙이기")
#root.geometry("100x100") 서로다른 크기, 위치에 있는것으로 레이어드 구성해서 이부분 삭제


##파일추가 파일다이얼로그는 갖다쓰면됨
def add_file():
    files = filedialog.askopenfilenames(title="여러개의 파일을 선택하세요",\
        filetypes=(("PNG 파일","*.png")("모든파일","*.*")),\
        initialdir="C:\workspace")  #r-Rowstring
        #askopenfilenames,여러개의 파일 선택할 수 있도록/initialdir 파일추가버튼누르면 뜨는 최초의 경로 - 지정

    #사용자가 선택한 파일 목록
    for file in files:
        list_file.insert(END, file)  #리스트프레임에 들어가도록

###파일 선택 삭제
def del_file():
    # print(list_file.curselection())
    # 파일 선택했을때 그 파일의 인덱스 reversed;뒤에서부터 가져오기
    for index in reversed(list_file.curselection()):
        list_file.delete(index)

##저장 경로 (폴더) 기능
def browse_dest_path():
    folder_selected = filedialog.askdirectory()
    if folder_selected is None:  #사용자가 취소를 누룰 때, 취소 : None
        return
    #print(folder_selected) -> 폴더경로 확인, 눌렀을때랑 취소눌렀을때 출력으로 확인
    e_dest_path.delete(0,END)
    e_dest_path.insert(0, folder_selected)

###이미지 통합
def merge_image():
    #print(list_file.get(0,END)) #모든 파일 목록 가지고 오기 -확인
    images = [Image.open(x) for x in list_file.get(0,END)] #리스트파일에있는 모든 내용을 불러와서 열고 하나씩 images 리스트에 넣음
    # size -> size[0] : width/ size[1] : height 
    # 이미지 객체의 사이즈 번호 여러개의 이미지값 중 가장 넓은 width를 처리하고 모든 이미지의 전체 height 를 알기위해 값 받아와야함
    
    widths, heights = zip(*(x.size for x in images))
    # widths = [x.size[0] for x in images]
    # heights = [x.size[1] for x in images] --위의 zip 이용해수 두작업 한번에 할 수 있음
    # print("width : ", widths) -- 확인
    # print("heights : ", heights)


    #사진을 다 넣는 긴 스케치북 준비하기 위해 최대 가로 길이와 높이들의 합을 구함.
    max_width, total_height = max(widths), sum(heights)
    # print("max_width : ", max(widths)) -- 확인
    # print("heigtotal_height : ", sum(heights))

    #빈 스케치북 만들기
    result_img = Image.new("RGB",(max_width, total_height),(255,255,255)) #RGB 배경 흰색 255
    #x는 그대론데 y를 기준으로 하나씩 추가하면서 이미지 늘려가기위해 쓰는 변수
    y_offset = 0
    # for img in images:
    #     result_img.paste(img, (0,y_offset))
    #     y_offset += img.size[1]   # height 값(y 값) 더해준다

    for idx, img in enumerate(images):
        result_img.paste(img, (0,y_offset))
        y_offset += img.size[1]

        progress = (idx + 1)/len(images) * 100 #실제 percent 정보를 계산
        p_var.set(progress)
        progress_bar.update()  #ui 업데이트

        
    #실제 실행, 실제로 이미지가 저장되는 경로    
    dest_path = os.path.join(e_dest_path.get(), "nado_photo.jpg")
    result_img.save(dest_path)
    msgbox.showinfo("알림","작업이 완료되었습니다.")


##시작버튼 기능
def start():
    #각 옵션들 값을 확인
    print("가로넓이: ", cmb_width.get())
    print("간격: ", cmb_space.get())
    print("포맷 :", cmb_format.get())

    #파일 목록 확인, 파일이 하나도 없을때
    if list_file.size() == 0:
        msgbox.showwarning("경고","이미지 파일을 추가하세요.")
        return

    #저장 경로 확인,저장경로가 비어있을 경우
    if len(e_dest_path.get()) == 0:
        msgbox.showwarning("경고","저장 경로를 선택하세요.")
        return

    #이미지 통합 작업
    merge_image()



#파일 프레임, 맨 위의 (파일 추가, 선택 삭체 )
file_frame = Frame(root)
file_frame.pack(fill='x', pady=5, padx=5) #padx,y 간격띄우기

btn_add_file = Button(file_frame, padx = 5, pady = 5, width = 12, text="파일 추가",command=add_file)
btn_add_file.pack(side="left")
btn_del_file = Button(file_frame, padx = 5, pady = 5, width = 12, text="파일 삭제",command=del_file)
btn_del_file.pack(side="right")


#리스트 프레임
list_frame = Frame(root)
list_frame.pack(fill="both", pady=5, padx=5)

#리스트 스크롤바
scr= Scrollbar(list_frame)
scr.pack(side="right", fill="y")

list_file = Listbox(list_frame, selectmode="extended", height=15, yscrollcommand=scr.set)
list_file.pack(side="left", fill="both", expand=True)
scr.config(command=list_file.yview)


#저장 경로 프레임
path_frame = LabelFrame(root, text="저장 경로")
path_frame.pack(fill='x', pady=5, padx=5, ipady=5)

e_dest_path = Entry(path_frame)
e_dest_path.pack(side="left", fill="x", expand=True, pady=5, padx=5, ipady=4) #expand=True 모든항목 보여주기 #ipady 안쪽 높이 영역

btn_dest_path = Button(path_frame, text = "찾아보기", width =10, command=browse_dest_path)
btn_dest_path.pack(side="right", pady=5, padx=5)


#옵션 프레임
frame_option = LabelFrame(root, text="옵션")
frame_option.pack(pady=5, padx=5, ipady=5)

#1. 가로 넓이 옵션
# 가로 넓이 레이블
lbl_width = Label(frame_option,text="가로넓이",width=8)
lbl_width.pack(side="left", pady=5, padx=5) #왼쪽으로 설정하지 않으면 아래로 쌓임. 왼쪽으로 설정하면 왼쪽에서 옆(오른쪽옆)으로 추가됨

# 가로 넓이 콤포
opt_width = ["원본유지", "1024", "800", "640"]
cmb_width = ttk.Combobox(frame_option, state="readonly", values=opt_width, width=10)
cmb_width.current(0)
cmb_width.pack(side="left", pady=5, padx=5)


#2. 간격 옵션
#간격 옵션 레이블
lbl_space = Label(frame_option,text="간격",width=8)
lbl_space.pack(side="left", pady=5, padx=5)

#간격 옵션 콤보
opt_space = ["없음", "좁게", "보통", "넓게"]
cmb_space = ttk.Combobox(frame_option, state="readonly", values=opt_space, width=10)
cmb_space.current(0)
cmb_space.pack(side="left", pady=5, padx=5)

#3. 파일 포맷 옵션
#파일 포맷 레이블
lbl_format = Label(frame_option,text="포맷",width=8)
lbl_format.pack(side="left", pady=5, padx=5)

#파일 포맷 콤보
opt_format = ["PNG", "JPG", "BMP"]
cmb_format = ttk.Combobox(frame_option, state="readonly", values=opt_format, width=10)
cmb_format.current(0)
cmb_format.pack(side="left", pady=5, padx=5)


#진행상황 프로그레스바
frame_progress = LabelFrame(root, text="진행상황")
frame_progress.pack(fill="x", pady=5, padx=5, ipady=5)

p_var = DoubleVar()
progress_bar = ttk.Progressbar(frame_progress,maximum=100,variable=p_var)
progress_bar.pack(fill="x", pady=5, padx=5)



#실행 프레임
frame_run = Frame(root)
frame_run.pack(fill="x", pady=5, padx=5)

btn_close = Button(frame_run,padx=5,pady=5, text="닫기",width=12,command=root.quit)
btn_close.pack(side="right", pady=5, padx=5)
btn_start = Button(frame_run,padx=5,pady=5, text="시작",width=12,command=start)
btn_start.pack(side="right", pady=5, padx=5)


root.resizable(False,False) #너비, 높이 창 크기 변경 불가
root.mainloop()


<>:19: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:19: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<ipython-input-2-8a76f0646255>:19: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  filetypes=(("PNG 파일","*.png")("모든파일","*.*")),\
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\anaconda3\envs\cakd3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-2-8a76f0646255>", line 19, in add_file
    filetypes=(("PNG 파일","*.png")("모든파일","*.*")),\
TypeError: 'tuple' object is not callable
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\anaconda3\envs\cakd3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-2-8a76f0646255>", line 19, in add_file
    filetypes=(("PNG 파일","*.png")("모든파일","*.*")),\
TypeError: 'tuple' object is not callable
